#### Import Libraries

In [1]:
import streamlit as st

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

#### Load Datasets

In [3]:
df_1 = pd.read_csv("dataset.csv")

print("Dataset shape is:", df_1.shape)

df_1.head()

Dataset shape is: (4920, 18)


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_2 = pd.read_csv("symptom_Description.csv")

print("Description shape is:", df_2.shape)

df_2.head()

Description shape is: (41, 2)


,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...
2,Allergy,An allergy is an immune system response to a f...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
4,Psoriasis,Psoriasis is a common skin disorder that forms...


In [39]:
df_2.columns

Index(['Disease', 'Description'], dtype='object')

In [40]:
d = "Allergy"
df_2[df_2["Disease"] == d]["Description"].item()

"An allergy is an immune system response to a foreign substance that's not typically harmful to your body.They can include certain foods, pollen, or pet dander. Your immune system's job is to keep you healthy by fighting harmful pathogens."

In [43]:
df_3 = pd.read_csv("symptom_precaution.csv")

print("Precaution shape is:", df_3.shape)
#df_3.fillna()
df_3.head()

Precaution shape is: (41, 5)


,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


In [41]:
print("Precautions to take are:")
print(df_3[df_3["Disease"] == d]["Precaution_1"].item()) 
print(df_3[df_3["Disease"] == d]["Precaution_2"].item())
print(df_3[df_3["Disease"] == d]["Precaution_3"].item())
print(df_3[df_3["Disease"] == d]["Precaution_4"].item())


Precautions to take are:
apply calamine
cover area with bandage
nan
use ice to compress itching


In [44]:
"""A function that takes in a disease name , gives you a discription about the disease and precautions to take."""
def drug_info(disease):
    d = disease

    # Print disease description
    print(df_2[df_2["Disease"] == d]["Description"].item())

    # Print Precautions to take
    print("Precautions to take are:")
    print(df_3[df_3["Disease"] == d]["Precaution_1"].item()) 
    print(df_3[df_3["Disease"] == d]["Precaution_2"].item())
    print(df_3[df_3["Disease"] == d]["Precaution_3"].item())
    print(df_3[df_3["Disease"] == d]["Precaution_4"].item())

In [45]:
drug_info("Malaria")

An infectious disease caused by protozoan parasites from the Plasmodium family that can be transmitted by the bite of the Anopheles mosquito or by a contaminated needle or transfusion. Falciparum malaria is the most deadly type.
Precautions to take are:
Consult nearest hospital
avoid oily food
avoid non veg food
keep mosquitos out


In [46]:
drug_info("Allergy")

An allergy is an immune system response to a foreign substance that's not typically harmful to your body.They can include certain foods, pollen, or pet dander. Your immune system's job is to keep you healthy by fighting harmful pathogens.
Precautions to take are:
apply calamine
cover area with bandage
nan
use ice to compress itching


In [54]:
df_4 = pd.read_csv("Symptom-severity.csv")

print("Severity shape is:", df_4.shape)

df_4.head()

Severity shape is: (133, 2)


,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [55]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Symptom  133 non-null    object
 1   weight   133 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


In [56]:
#for i in df_4.index:
 #   print(df_4['Symptom'][i], df_4['weight'][i])

In [57]:
#convert data_severity to dictionnary
data_dict = df_4.set_index('Symptom').T.to_dict()
data_dict

C:\Users\ThisPC\AppData\Local\Temp\ipykernel_8376\2212837546.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  data_dict = df_4.set_index('Symptom').T.to_dict()


{'itching': {'weight': 1},
 'skin_rash': {'weight': 3},
 'nodal_skin_eruptions': {'weight': 4},
 'continuous_sneezing': {'weight': 4},
 'shivering': {'weight': 5},
 'chills': {'weight': 3},
 'joint_pain': {'weight': 3},
 'stomach_pain': {'weight': 5},
 'acidity': {'weight': 3},
 'ulcers_on_tongue': {'weight': 4},
 'muscle_wasting': {'weight': 3},
 'vomiting': {'weight': 5},
 'burning_micturition': {'weight': 6},
 'spotting_urination': {'weight': 6},
 'fatigue': {'weight': 4},
 'weight_gain': {'weight': 3},
 'anxiety': {'weight': 4},
 'cold_hands_and_feets': {'weight': 5},
 'mood_swings': {'weight': 3},
 'weight_loss': {'weight': 3},
 'restlessness': {'weight': 5},
 'lethargy': {'weight': 2},
 'patches_in_throat': {'weight': 6},
 'irregular_sugar_level': {'weight': 5},
 'cough': {'weight': 4},
 'high_fever': {'weight': 7},
 'sunken_eyes': {'weight': 3},
 'breathlessness': {'weight': 4},
 'sweating': {'weight': 3},
 'dehydration': {'weight': 4},
 'indigestion': {'weight': 5},
 'headache'

In [58]:
df_4.head()

,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [59]:
#df_4.groupby("weight").value_counts().tail(20)

In [60]:
def remove_space_between_words(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip().str.replace(" ", "_")
    return df

In [61]:
df_1= remove_space_between_words(df_1)
df_1.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal_infection,itching,skin_rash,nodal_skin_eruptions,dischromic__patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal_infection,skin_rash,nodal_skin_eruptions,dischromic__patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal_infection,itching,nodal_skin_eruptions,dischromic__patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal_infection,itching,skin_rash,dischromic__patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal_infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
df_4.head()

,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [63]:
symps = {"Symptom_1": "vomiting", "Symptom_02" : np.nan, "Symptom_03" : np.nan,"Symptom_04" : np.nan,"Symptom_05" : np.nan,"Symptom_06" : np.nan,"Symptom_07" : np.nan,"Symptom_08" : np.nan,"Symptom_09" : np.nan,"Symptom_10" : np.nan,"Symptom_11" : np.nan, "Symptom_12" : np.nan,"Symptom_13" : np.nan,"Symptom_14" : np.nan,"Symptom_15" : np.nan, "Symptom_16" : np.nan,"Symptom_17" : np.nan,}

In [64]:
Test = pd.DataFrame(symps, index=range(1))
Test

,Symptom_1,Symptom_02,Symptom_03,Symptom_04,Symptom_05,Symptom_06,Symptom_07,Symptom_08,Symptom_09,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,vomiting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
def encode_symp(Test, df_4):
    for i in df_4.index:
        symptom = df_4["Symptom"][i]
        weight = df_4["weight"][i]
        Test = Test.replace(symptom, weight)
    
    # Replace missing values with 0
    Test = Test.fillna(0)

    # Additional hardcoded replacements
    Test = Test.replace("foul_smell_of_urine", 5)
    Test = Test.replace("dischromic__patches", 6)
    Test = Test.replace("spotting__urination", 6)
    return Test   

In [66]:
Test_01 = encode_symp(Test, df_4)
Test_01

,Symptom_1,Symptom_02,Symptom_03,Symptom_04,Symptom_05,Symptom_06,Symptom_07,Symptom_08,Symptom_09,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
def encode_symptoms(df, df_4):
    for i in df_4.index:
        symptom = df_4["Symptom"][i]
        weight = df_4["weight"][i]
        df = df.replace(symptom, weight)

    # Replace missing values with 0
    df = df.fillna(0)

    # Additional hardcoded replacements
    df = df.replace("foul_smell_of_urine", 5)
    df = df.replace("dischromic__patches", 6)
    df = df.replace("spotting__urination", 6)
    
    return df

In [68]:
new_df_1 = encode_symptoms(df_1, df_4)
new_df_1.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal_infection,1,3,4,6,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Fungal_infection,3,4,6,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Fungal_infection,1,4,6,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Fungal_infection,1,3,6,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Fungal_infection,1,3,4,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
new_df_1.describe()

,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
count,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000,4920.000000
mean,3.410976,4.165854,4.173171,3.967073,3.195122,2.654878,2.059756,1.712195,1.554878,1.304878,0.979268,0.702439,0.492683,0.323171,0.282927,0.136585,0.029268
std,1.316310,1.207541,1.249229,1.855343,2.163883,2.384472,2.340259,2.244483,2.250034,2.061402,1.880334,1.799707,1.529126,1.305543,1.298518,0.719747,0.240191
min,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,4.000000,4.000000,4.000000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.000000,4.000000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,6.000000,7.000000,7.000000,6.000000,7.000000,7.000000,6.000000,7.000000,7.000000,5.000000,2.000000


In [70]:
new_df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Disease     4920 non-null   object 
 1   Symptom_1   4920 non-null   int64  
 2   Symptom_2   4920 non-null   int64  
 3   Symptom_3   4920 non-null   int64  
 4   Symptom_4   4920 non-null   int64  
 5   Symptom_5   4920 non-null   int64  
 6   Symptom_6   4920 non-null   float64
 7   Symptom_7   4920 non-null   float64
 8   Symptom_8   4920 non-null   float64
 9   Symptom_9   4920 non-null   float64
 10  Symptom_10  4920 non-null   float64
 11  Symptom_11  4920 non-null   float64
 12  Symptom_12  4920 non-null   float64
 13  Symptom_13  4920 non-null   float64
 14  Symptom_14  4920 non-null   float64
 15  Symptom_15  4920 non-null   float64
 16  Symptom_16  4920 non-null   float64
 17  Symptom_17  4920 non-null   float64
dtypes: float64(12), int64(5), object(1)
memory usage: 692.0+ KB


In [71]:
names = []

# Iterate through columns except for "Disease"
for col in new_df_1.columns:
    if col != "Disease":
        # Iterate through rows in the column
        for symptom in new_df_1[col]:
            # Check if the value is a string and not in the 'names' list
            if isinstance(symptom, str) and symptom not in names:
                names.append(symptom)

# Check if all symptoms have been replaced
all_replaced = all(symptom not in names for symptom in df_4["Symptom"])

if all_replaced:
    print("All symptoms have been replaced.")
else:
    print("The following symptoms were not replaced:", names)

All symptoms have been replaced.


In [72]:
# separating the data and labels
X = new_df_1.drop(columns='Disease', axis=1)
Y = new_df_1['Disease']

In [73]:
X

,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,1,3,4,6,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,4,6,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,4,6,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,6,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,3,4,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,5,3,5,6,4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4916,3,2,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4917,6,4,5,6,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4918,3,3,3,2,2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Data Standardization

In [74]:
scaler = StandardScaler()

In [75]:
scaler.fit(X)

StandardScaler()

In [76]:
standardized_data = scaler.transform(X)

In [35]:
print(standardized_data)

[[-1.83180372 -0.96557578 -0.13863618 ... -0.21790657 -0.18978799
  -0.12186667]
 [-0.3122498  -0.13736225  1.46251409 ... -0.21790657 -0.18978799
  -0.12186667]
 [-1.83180372 -0.13736225  1.46251409 ... -0.21790657 -0.18978799
  -0.12186667]
 ...
 [ 1.96708109 -0.13736225  0.66193895 ... -0.21790657 -0.18978799
  -0.12186667]
 [-0.3122498  -0.96557578 -0.93921132 ... -0.21790657 -0.18978799
  -0.12186667]
 [-0.3122498   2.34727837 -0.13863618 ... -0.21790657 -0.18978799
  -0.12186667]]


In [77]:
X = standardized_data
Y = new_df_1['Disease']

In [78]:
print(X)

[[-1.83180372 -0.96557578 -0.13863618 ... -0.21790657 -0.18978799
  -0.12186667]
 [-0.3122498  -0.13736225  1.46251409 ... -0.21790657 -0.18978799
  -0.12186667]
 [-1.83180372 -0.13736225  1.46251409 ... -0.21790657 -0.18978799
  -0.12186667]
 ...
 [ 1.96708109 -0.13736225  0.66193895 ... -0.21790657 -0.18978799
  -0.12186667]
 [-0.3122498  -0.96557578 -0.93921132 ... -0.21790657 -0.18978799
  -0.12186667]
 [-0.3122498   2.34727837 -0.13863618 ... -0.21790657 -0.18978799
  -0.12186667]]


In [38]:
print(Y)

0                              Fungal_infection
1                              Fungal_infection
2                              Fungal_infection
3                              Fungal_infection
4                              Fungal_infection
                         ...                   
4915    (vertigo)_Paroymsal__Positional_Vertigo
4916                                       Acne
4917                    Urinary_tract_infection
4918                                  Psoriasis
4919                                   Impetigo
Name: Disease, Length: 4920, dtype: object


####  Random Forest Classifier

In [79]:
from sklearn.ensemble import RandomForestClassifier
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create a Random Forest Classifier
rfc_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model on the training data
rfc_classifier.fit(X_train, Y_train)

# Predict disease labels on the testing data
Y_pred = rfc_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(Y_test, Y_pred)

precision = precision_score(Y_test, Y_pred, average='macro')
recall = recall_score(Y_test, Y_pred,average='macro')
f1 = f1_score(Y_test, Y_pred, average='macro')

print("Random Forest")
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print(accuracy_score(Y_test, Y_pred,normalize=False))
print("Confusion matrix")
conf_matrix=confusion_matrix(Y_test,Y_pred)
print(conf_matrix)

Random Forest
Accuracy: 0.991869918699187
Precision: 0.9928005598737307
Recall: 0.9909757027776227
F1-Score: 0.9914564591749411
976
Confusion matrix
[[18  0  0 ...  0  0  0]
 [ 0 30  0 ...  0  0  0]
 [ 0  0 24 ...  0  0  0]
 ...
 [ 0  0  0 ... 26  0  0]
 [ 0  0  0 ...  0 22  0]
 [ 0  0  0 ...  0  0 34]]


#### Create Model that predicts disease and returns information about the disease

In [83]:
Test_01

,Symptom_1,Symptom_02,Symptom_03,Symptom_04,Symptom_05,Symptom_06,Symptom_07,Symptom_08,Symptom_09,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
standardized_test = scaler.transform(Test_01)

C:\Users\ThisPC\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Symptom_02
- Symptom_03
- Symptom_04
- Symptom_05
- Symptom_06
- ...
Feature names seen at fit time, yet now missing:
- Symptom_2
- Symptom_3
- Symptom_4
- Symptom_5
- Symptom_6
- ...

  warnings.warn(message, FutureWarning)


In [86]:
drug_info(rfc_classifier.predict(standardized_test).item())

Acne vulgaris is the formation of comedones, papules, pustules, nodules, and/or cysts as a result of obstruction and inflammation of pilosebaceous units (hair follicles and their accompanying sebaceous gland). Acne develops on the face and upper trunk. It most often affects adolescents.
Precautions to take are:
bath twice
avoid fatty spicy food
drink plenty of water
avoid too many products
